<a href="https://colab.research.google.com/github/keziammanuela/KomputasiIntelegensiaTasks/blob/main/TaskWeek7.Sentiment%20Analysis%20Model%20from%20Hugging%20Face.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Nama: Kezia Emmanuela Bangun

NPM: 2006572964

# Sentiment Analysis Model

In [ ]:
# First, install the transformers library if you haven't already
!pip install transformers

# Import necessary libraries
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F

# Load the model and tokenizer
model_name = "finiteautomata/bertweet-base-sentiment-analysis"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Function to get sentiment
def analyze_sentiment(text):
    # Tokenize the text
    inputs = tokenizer(text, return_tensors="pt")

    # Get the model outputs
    with torch.no_grad():
        outputs = model(**inputs)

    # Use softmax to get probabilities for each class
    probs = F.softmax(outputs.logits, dim=-1)

    # Map the predictions to sentiment labels
    labels = ['negative', 'neutral', 'positive']
    sentiment = labels[torch.argmax(probs).item()]
    confidence = torch.max(probs).item()

    return sentiment, confidence

# Test the function
text = "I'm so happy to be using this model for sentiment analysis!"
sentiment, confidence = analyze_sentiment(text)
print(f"Sentiment: {sentiment}, Confidence: {confidence:.2f}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/338 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/843k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/22.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/949 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/540M [00:00<?, ?B/s]

Sentiment: positive, Confidence: 0.99


# Predicting Sentiment of a Sentence

In [ ]:
# Function to test the model with new text
def test_sentiment():
    text = input("Enter text for sentiment analysis: ")
    sentiment, confidence = analyze_sentiment(text)
    print(f"Sentiment: {sentiment}, Confidence: {confidence:.2f}")

# Run the test function
test_sentiment()

Enter text for sentiment analysis: Moodeng lives in Thailand
Sentiment: neutral, Confidence: 0.97


In [ ]:
# Function to test the model with new text
def test_sentiment():
    text = input("Enter text for sentiment analysis: ")
    sentiment, confidence = analyze_sentiment(text)
    print(f"Sentiment: {sentiment}, Confidence: {confidence:.2f}")

# Run the test function
test_sentiment()

Enter text for sentiment analysis: I hate you, but I miss you
Sentiment: negative, Confidence: 0.88


# Checking Model's Accuracy Using a Synthetic Dataset

In [ ]:
!pip install datasets transformers
!pip install emoji==0.6.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-0.6.0-py3-none-any.whl size=49719 sha256=eafd99f42a43099c075bc9925329d2c76875ad1268d172474cc3d30cf62deaa2
  Stored in directory: /root/.cache/pip/wheels/1b/bd/d9/310c33c45a553798a714e27e3b8395d37128425442b8c78e07
Successfully built emoji


In [ ]:
from datasets import load_dataset
import random
import emoji
print(emoji.emojize('Python is :thumbs_up:'))  # Output: Python is 👍

# Load the SST-5 dataset (Stanford Sentiment Treebank)
dataset = load_dataset("sst", "default")

# Mapping the labels to 'positive', 'neutral', 'negative' based on SST-5 labels
# SST-5 labels: 0 (very negative), 1 (negative), 2 (neutral), 3 (positive), 4 (very positive)
label_map = {0: "negative", 1: "negative", 2: "neutral", 3: "positive", 4: "positive"}

# Sample a smaller subset (e.g., 1000 samples from the test set)
sample_data = random.sample(list(dataset["test"]), 1000)

# Load the model and tokenizer
model_name = "finiteautomata/bertweet-base-sentiment-analysis"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Function to get sentiment prediction
def analyze_sentiment(text):
    # Tokenize the text
    inputs = tokenizer(text, return_tensors="pt")

    # Get model predictions
    with torch.no_grad():
        outputs = model(**inputs)

    # Get probabilities for each class
    probs = F.softmax(outputs.logits, dim=-1)

    # Get the label with the highest probability
    labels = ['negative', 'neutral', 'positive']
    sentiment = labels[torch.argmax(probs).item()]
    confidence = torch.max(probs).item()

    return sentiment, confidence

# Calculate accuracy on the sample
def calculate_accuracy(sample_data):
    correct_predictions = 0
    total_samples = len(sample_data)

    for item in sample_data:
        text = item["sentence"]

        # Convert the label to integer if it's a float. This handles cases
        # where the dataset returns floats instead of integers for labels.
        # Round to the nearest integer before accessing label_map
        true_label_index = int(round(item["label"]))
        true_label = label_map[true_label_index]

        # Get model's prediction
        predicted_label, _ = analyze_sentiment(text)

        # Compare with the true label
        if predicted_label == true_label:
            correct_predictions += 1

    # Calculate accuracy
    accuracy = correct_predictions / total_samples
    return accuracy

# Calculate and display accuracy
accuracy = calculate_accuracy(sample_data)
print(f"Model Accuracy on SST-5 Sample: {accuracy * 100:.2f}%")

Python is 👍
Model Accuracy on SST-5 Sample: 33.60%


Diperoleh Akurasi Model pada Sampel Dataset SST-5 adalah 33.60%.

# Implement Attention Transformer on Pretrained Model

In [ ]:
class AttentionTransformer(nn.Module):
    def __init__(self, model_name):
        super(AttentionTransformer, self).__init__()
        self.model = AutoModel.from_pretrained(model_name)
        self.attention = nn.MultiheadAttention(embed_dim=self.model.config.hidden_size, num_heads=8)
        self.fc = nn.Linear(self.model.config.hidden_size, self.model.config.num_labels)

    def forward(self, input_ids, attention_mask):
        # Get the outputs from the pre-trained model
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)

        # Extract the last hidden state
        hidden_states = outputs.last_hidden_state  # Shape: (batch_size, seq_length, hidden_size)

        # Apply attention
        attn_output, _ = self.attention(hidden_states, hidden_states, hidden_states)

        # Use the output from the attention layer
        # Here, we can either take the mean of the attention outputs or use the CLS token
        cls_output = attn_output[:, 0, :]  # Take the output corresponding to the [CLS] token

        # Pass through the fully connected layer
        logits = self.fc(cls_output)

        return logits

# Initialize the model and tokenizer
model_name = "finiteautomata/bertweet-base-sentiment-analysis"
attention_model = AttentionTransformer(model_name)

# Example of how to use the model
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Sample input
sample_text = "This is a great model for sentiment analysis."
inputs = tokenizer(sample_text, return_tensors="pt")

# Forward pass
with torch.no_grad():
    outputs = attention_model(inputs['input_ids'], inputs['attention_mask'])

print(outputs)  # Logits for sentiment classification

Some weights of RobertaModel were not initialized from the model checkpoint at finiteautomata/bertweet-base-sentiment-analysis and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tensor([[ 0.0593,  0.0345, -0.0145]])


In [ ]:
import torch.nn.functional as F

logits = torch.tensor([[0.0593, 0.0345, -0.0145]])
probabilities = F.softmax(logits, dim=-1)

print(probabilities)

tensor([[0.3443, 0.3359, 0.3198]])


# Checking Model's Accuracy After Using Attention Transformer Using The Same Synthetic Dataset and Compare

In [ ]:
import random
import pandas as pd
import torch
import torch.nn.functional as F
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModel

# Load the SST-5 dataset (Stanford Sentiment Treebank)
dataset = load_dataset("sst", "default")

# Mapping the labels: 0 (very negative), 1 (negative), 2 (neutral), 3 (positive), 4 (very positive)
label_map = {0: "negative", 1: "negative", 2: "neutral", 3: "positive", 4: "positive"}

# Convert the dataset into a DataFrame for easier manipulation
df = pd.DataFrame(dataset['train'])

# Sample a smaller subset (e.g., 500 samples)
sample_data = df.sample(500, random_state=42)

# Define the Attention Transformer class
class AttentionTransformer(nn.Module):
    def __init__(self, model_name):
        super(AttentionTransformer, self).__init__()
        self.model = AutoModel.from_pretrained(model_name)
        self.attention = nn.MultiheadAttention(embed_dim=self.model.config.hidden_size, num_heads=8)
        self.fc = nn.Linear(self.model.config.hidden_size, 5)  # Adjust output size for 5 classes

    def forward(self, input_ids, attention_mask):
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
        hidden_states = outputs.last_hidden_state
        attn_output, _ = self.attention(hidden_states, hidden_states, hidden_states)
        cls_output = attn_output[:, 0, :]  # Use the [CLS] token output
        logits = self.fc(cls_output)
        return logits

# Initialize the Attention Transformer model
model_name = "finiteautomata/bertweet-base-sentiment-analysis"
attention_model = AttentionTransformer(model_name)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Function to get sentiment prediction
def analyze_sentiment(attention_model, text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        logits = attention_model(inputs['input_ids'], inputs['attention_mask'])
    return logits

# Calculate accuracy on the sampled dataset
def calculate_accuracy(attention_model, sample_data):
    correct_predictions = 0
    total_samples = len(sample_data)

    for index, row in sample_data.iterrows():
        text = row['sentence']  # Assuming the column for text is 'sentence'
        true_label = int(row['label'])  # Convert label to integer for mapping
        true_label = label_map[true_label]

        # Get model's prediction
        logits = analyze_sentiment(attention_model, text)
        predicted_label_index = torch.argmax(logits, dim=1).item()
        predicted_label = label_map[predicted_label_index]

        # Compare with the true label
        if predicted_label == true_label:
            correct_predictions += 1

    # Calculate accuracy
    accuracy = correct_predictions / total_samples
    return accuracy

# Calculate and display accuracy
accuracy_attention_model = calculate_accuracy(attention_model, sample_data)
print(f"Attention Transformer Model Accuracy on SST-5 Sample: {accuracy_attention_model * 100:.2f}%")

Some weights of RobertaModel were not initialized from the model checkpoint at finiteautomata/bertweet-base-sentiment-analysis and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Attention Transformer Model Accuracy on SST-5 Sample: 71.00%


Akurasi Model setelah diterapkan Attention Transformer dengan sampel dataset SST-5 adalah 71%.

Setelah menerapkan model Attention Transformer, akurasi analisis sentimen meningkat dari 33.60% menjadi 71%. Hal ini menunjukkan bahwa mekanisme Attention sangat membantu dalam memahami konteks dan emosi dalam teks. Model dasar yang sebelumnya digunakan tidak mampu menangkap nuansa dan makna yang lebih dalam dari kalimat, sehingga akurasinya rendah. Dengan penambahan Attention, model bisa lebih fokus pada bagian-bagian penting dari kalimat, yang membantu dalam membedakan antara sentimen positif, negatif, dan netral. Selain itu, arsitektur Attention Transformer yang lebih kompleks memungkinkan model untuk menghasilkan representasi yang lebih baik dari teks, sehingga dapat mengenali pola yang lebih rumit dalam data. Dengan demikian, penggunaan Attention Transformer terbukti efektif dalam meningkatkan akurasi analisis sentimen, terutama pada dataset seperti SST-5 yang memiliki nuansa emosional yang kaya.